In [ ]:
import os
import re
import sys
from pathlib import Path
from os import path
import json
import time
import pyodbc
import webbrowser
import shutil
import csv
import pandas as pd
import folium
import datetime
import sklearn
from sklearn.neighbors import KNeighborsClassifier
import pyodbc
import numpy as np
import math
import datetime
import warnings
warnings.simplefilter('ignore')

# ヒュベニの公式
# https://butter-tiger.hatenablog.com/entry/2020/08/20/222650
POLE_RADIUS = 6356752  # 極半径(短半径)
EQUATOR_RADIUS = 6378137  # 赤道半径(長半径)
E = 0.081819191042815790  # 離心率
E2 = 0.006694380022900788  # 離心率の２乗

<font size = 6 color =red>ここからは必要な関数を定義する↓</font>  
  
<font size = 3>この部分はisobe_arinaga_map\config.py</font>  

In [ ]:
#データベース接続
driver='{SQL Server}'
server = '*******************'
database = '*******************'
uid = '*******************'
pwd = '*******************'
trusted_connection ='yes'

AltitudeAllTableName = '[*******************].[dbo].[ALTITUDE_10M_MESH]'
AltitudeTableName = '[*******************].[dbo].[ALTITUDE_10M_MESH_REGISTERED_FIXED]' #通勤ルート用に補正したテーブル？
EfficiencyTableName = '[*******************].[dbo].[EFFICIENCY_MAP]'
EfficiencyMaxTableName = '[*******************].[dbo].[EFFICIENCY_MAP_MAX]'
Cars_Tablename = '[*******************].[dbo].[CARS]'
Place_Tablename = '[*******************].[dbo].[PLACE]'

minTorque = 76  #EfficiencyMaxTableNameテーブルのTorqueカラムの最小値
maxTorque = 280 #EfficiencyMaxTableNameテーブルのTorqueカラムの最大値
maxRev = 9990   #EfficiencyMaxTableNameテーブルのRevカラムの最大値

def select_execute(con, sql):
    cursor = con.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    columnNames = [column[0] for column in cursor.description]
    cursor.close()
    return rows, columnNames

#https://www.codegrepper.com/code-examples/python/pyodbc.row+to+dictionary  sql -> dict変換の参考に
class CarModel():
    def __init__(self, car_id):
        connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';UID='+uid+';PED='+pwd+';DATABASE='+database+';Trusted_Connection='+trusted_connection+';')

        sql = (
            'SELECT CAR_ID, \
            MODEL, \
            WEIGHT, \
            TIRE_RADIUS, \
            REDUCTION_RATIO, \
            CD_VALUE, \
            FRONTAL_PROJECTED_AREA \
            FROM ' + Cars_Tablename
            + 'WHERE CAR_ID = ' + str(car_id)
        )
        self.rows, self.column_Names = select_execute(connect, sql)
        connect.close()

    def get_car_model(self):
        
        dict_carModel = {}
        column_Names2 = ['CarID', 'Model', 'CarWeight', 'TireRadius', 'ReductionRatio', 'CdValue', 'FrontalProjectedArea']

        if len(self.rows) == 0:
            print('CarIDが違います')
            sys.exit()
        elif len(self.rows) == 1:
            print(self.rows[0])
            print(type(self.rows[0]))
            dict_carModel.update( dict( zip( column_Names2 , self.rows[0] ) ) )
            dict_carModel['Weight'] = dict_carModel['CarWeight'] + CarOtherWeight
        else:
            print('CarID重複エラー？？')
        return dict_carModel

def get_max_driving_force(weight):
    return -0.15 * GravityResistanceCoefficient * weight



def isInPlace(lat, long, place_ID):
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';UID='+uid+';PED='+pwd+';DATABASE='+database+';Trusted_Connection='+trusted_connection+';')
    sql = (
            'SELECT START_LATITUDE, \
            END_LATITUDE, \
            START_LONGITUDE, \
            END_LONGITUDE \
            FROM ' + Place_Tablename
            + ' WHERE PLACE_ID = ' + str(place_ID)
        )
    rows, column_Names = select_execute(connect, sql)
    dict_place = {}
    if len(rows) == 0:
        print('PlaceIDが違います')
        sys.exit()
    elif len(rows) == 1:
        print(rows[0])
        dict_place.update( dict( zip( column_Names , rows[0] ) ) )
        if dict_place['START_LATITUDE'] < lat and lat < dict_place['END_LATITUDE'] and dict_place['START_LONGITUDE'] < long and long < dict_place['END_LONGITUDE']:
            return 'true'
    else:
        print('CarID重複エラー？？')


    connect.close()

    return 'false'



#環境データ
myu = 0.015
rho = 1.22
GravityResistanceCoefficient = 9.80665
windSpeed = 0.0

#車両データ
'''
Weight = 1600
FrontProjectedArea = 2.19
CdValue = 0.28
TireRadius = 0.3155
ReductionRatio = 7.9377
'''
'''
Weight = 1760
FrontProjectedArea = 2.24
CdValue = 0.28
TireRadius = 0.3234
ReductionRatio = 8.1938
'''
InverterEfficiency = 0.95
MaxDrivingPower = -30
#MaxDrivingForce = -0.15 * GravityResistanceCoefficient * Weight
CarOtherWeight = 80

<font size = 3>この部分はisobe_arinaga_map\MathUtil.py</font>  

In [ ]:
#座標の系列データを入力とし、高度の系列データを出力するプログラム
def altDao(gpsRows):
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';UID='+uid+';PED='+pwd+';DATABASE='+database+';Trusted_Connection='+trusted_connection+';')
    altitudeList = []
    for i in gpsRows:
        _lat = str(i[0])
        _long = str(i[1])
        sql = 'SELECT ALTITUDE FROM ' + AltitudeTableName + ' WHERE LOWER_LATITUDE <= ' + _lat + ' AND UPPER_LATITUDE > ' + _lat + ' AND LOWER_LONGITUDE <= ' + _long + ' AND UPPER_LONGITUDE > ' + _long
        
        rows = select_execute(connect, sql)
        if len(rows) > 0:
            altitude = round(float(str(rows[0]).replace(', )', '').replace('(', '')), 3)
        else:   #FIXEDのテーブルになかった場合
            sql = 'SELECT ALTITUDE FROM ' + AltitudeAllTableName + ' WHERE LOWER_LATITUDE <= ' + _lat + ' AND UPPER_LATITUDE > ' + _lat + ' AND LOWER_LONGITUDE <= ' + _long + ' AND UPPER_LONGITUDE > ' + _long
            rows = select_execute(connect, sql)
            if len(rows) > 0:
                altitude = round(float(str(rows[0]).replace(', )', '').replace('(', '')), 3)
            else:
                altitude = -99
                print(str(i) + '高度データを取得できませんでした')
        #print(str(i) + ' -> ' + str(altitude))
        altitudeList.append(altitude)
    print('--- altitude insert finished ---\n')
    connect.close()
    #print(altitudeList)
    return altitudeList


def Calc_Heading(_lat1, _long1, _lat2, _long2):
    heading = 0
    lat1 = _lat1 * math.pi / 180
    long1 = _long1 * math.pi / 180
    lat2 = _lat2 * math.pi / 180
    long2 = _long2 * math.pi / 180

    y = math.cos(lat2) * math.sin(long2 - long1)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(long2 - long1)
    heading = math.atan2(y, x) * 180 / math.pi
    if heading < 0:
        heading = heading + 360

    return heading

def select_execute(con, sql):
    cursor = con.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    cursor.close()
    return rows

def get_efficiency(speed, torque, dict_carModel):

    #print('---最大効率の処理を入れる必要あり---')
    efficiency = -1
    rpm = speed * 60 / (dict_carModel['TIRE_RADIUS'].iloc[0] * 2 * math.pi) * dict_carModel['REDUCTION_RATIO'].iloc[0]
    rev = math.floor(rpm / 10) * 10 #四捨五入ではなく切り捨て
    #print('rev:', rev, '  torque:', torque)
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';UID='+uid+';PED='+pwd+';DATABASE='+database+';Trusted_Connection='+trusted_connection+';')

    if rpm <= maxRev and torque <= maxTorque and torque >= minTorque:   #C#のSensorLogInserterはここの条件が逆になってる？？
        sql = 'SELECT EFFICIENCY FROM ' + EfficiencyMaxTableName + ' WHERE torque = ' + str(torque) + ' AND REV = ' + str(rev)
        rows = select_execute(connect, sql)
        if len(rows) > 0:
            efficiency = int(str(rows[0]).replace(', )', '').replace('(', ''))
            #print(efficiency, '  MAX')
    if efficiency == -1:
#         print('rpm: ',rpm)
#         print('rev: ',rev)
        sql = 'SELECT EFFICIENCY FROM ' + EfficiencyTableName + ' WHERE torque = ' + str(torque) + ' AND REV = ' + str(rev)
        rows = select_execute(connect, sql)
        if len(rows) > 0:
            efficiency = int(str(rows[0]).replace(', )', '').replace('(', ''))
            #print(efficiency, '  not MAX')
    if efficiency == -1:
        print('rpm: ',rpm)
        print('rev: ',rev)
        efficiency = 75
        print('変換効率取得で想定外の分岐が発生')
    #print(efficiency)
    connect.close()
    return efficiency

def Calc_RegeneEnergy(drivingPower, speed, efficiency, weight):
    regeneEnergy = 0
    if speed < 7 / 3.6:
        drivingForce = 0
    else:
        drivingForce = drivingPower * 1000 * 3600 / speed   #制動力[N]
    speedC = MaxDrivingPower * 1000 / get_max_driving_force(weight)   #限界回生力と限界回生エネルギーの時の回生力の低い方が変わるときの車速[m/s]
    if drivingPower < 0:
        if speed < 7 / 3.6:
            regeneEnergy = 0
        elif drivingPower * 3600 >= MaxDrivingPower and drivingForce >= get_max_driving_force(weight):
            regeneEnergy = drivingPower * efficiency
        elif speed <= speedC and drivingForce < get_max_driving_force(weight):
            regeneEnergy = get_max_driving_force(weight) * speed * efficiency / 3600 / 1000
        elif drivingPower * 3600 < MaxDrivingPower and speed > speedC:
            regeneEnergy = MaxDrivingPower / 3600 * efficiency
        else:
            regeneEnergy = 0
            print('回生制約で想定外の分岐が発生  speed:', str(speed), '  power:', str(drivingPower), '  efficiency:', str(efficiency))
        regeneEnergy = regeneEnergy / 100 * InverterEfficiency
        if(drivingPower > regeneEnergy):
            print("ドライビングパワーよりも回生エネルギーの方が大きくなっている，至急確認せよ")
            print(drivingPower, '  ', speed, '  ', efficiency, '  ', weight)
    return regeneEnergy

#ヒュベニの公式
#https://butter-tiger.hatenablog.com/entry/2020/08/20/222650
POLE_RADIUS = 6356752    # 極半径(短半径)
EQUATOR_RADIUS = 6378137 # 赤道半径(長半径)
E = 0.081819191042815790 # 離心率
E2= 0.006694380022900788 # 離心率の２乗
def distance(_lat1, _long1, _lat2, _long2):
    lat1 = math.radians(_lat1)
    long1 = math.radians(_long1)
    lat2 = math.radians(_lat2)
    long2 = math.radians(_long2)
    m_lat = (lat1 + lat2) / 2 # 平均緯度
    d_lat = abs(lat1 - lat2) # 緯度差
    d_lon = abs(long1 - long2) # 経度差
    W = math.sqrt(1-E2*math.pow(math.sin(m_lat),2))
    M = EQUATOR_RADIUS*(1-E2) / math.pow(W, 3) # 子午線曲率半径
    N = EQUATOR_RADIUS / W # 卯酉線曲率半径
    # d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2) + math.pow(point_a.altitude-point_b.altitude,2))
    d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2))
    return d

In [ ]:
def load_json_to_dict_UTF8(input_jsonpath):
    fp = open(input_jsonpath, "r", encoding="utf-8_sig")
    json_load = json.load(fp)
    return json_load

def distance(_lat1, _long1, _lat2, _long2):
    lat1 = math.radians(_lat1)
    long1 = math.radians(_long1)
    lat2 = math.radians(_lat2)
    long2 = math.radians(_long2)
    m_lat = (lat1 + lat2) / 2  # 平均緯度
    d_lat = abs(lat1 - lat2)  # 緯度差
    d_lon = abs(long1 - long2)  # 経度差
    W = math.sqrt(1 - E2 * math.pow(math.sin(m_lat), 2))
    M = EQUATOR_RADIUS * (1 - E2) / math.pow(W, 3)  # 子午線曲率半径
    N = EQUATOR_RADIUS / W  # 卯酉線曲率半径
    # d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2) + math.pow(point_a.altitude-point_b.altitude,2))
    d = math.sqrt(math.pow(M * d_lat, 2) + math.pow(N * d_lon * math.cos(m_lat), 2))
    return d


# convert JSON to [Latitude, Longitude, *******************_Elevation]

def ADASListGenerator_to_df(semantic_link_id,json_*******************_filepath):
    json_semanticlinkID_name = semantic_link_id 


    data_json = pd.DataFrame({'SemanticLink':[],
                              'Latitude':[],
                              'Longitude':[],
                              '*******************_Elevation':[],
                              'adas_null':[]})

    ## データ抽出
    *******************_response_json_list = []
    *******************_response_json_list = load_json_to_dict_UTF8(json_*******************_filepath)  # jsonの大外がリスト構造
    global adas_null 
    

    for i in range(len(load_json_to_dict_UTF8(json_*******************_filepath))):
        for j in range(len(load_json_to_dict_UTF8(json_*******************_filepath)[i]['result']['path'])):
            if 'adas' in load_json_to_dict_UTF8(json_*******************_filepath)[i]['result']['path'][j]['matchLink']:
                adas_null = 0
                if str(load_json_to_dict_UTF8(json_*******************_filepath)[i]['result']['path'][j]['matchLink']['adas']) == 'None':
                    adas_null = 1
                    continue
                roadelevation = load_json_to_dict_UTF8(json_*******************_filepath)[i]['result']['path'][j]['matchLink']['adas']['roadelevation']
                for h in range(len(roadelevation)):
                    adasPoint = load_json_to_dict_UTF8(json_*******************_filepath)[i]['result']['path'][j]['matchLink']['adas']['roadelevation'][h]
                    data_json.loc[len(data_json)+1] = [json_semanticlinkID_name, adasPoint["lat"], adasPoint["lon"], adasPoint["elevation"], adas_null]

    data_json.reset_index(drop = True,inplace = True)
    return data_json

In [ ]:
def get_data_from_SEGMENTS_ZDC(semantic_link_id ,segment_distance):
    driver='{SQL Server}'
    server = '*******************'
    database = '*******************'
    trusted_connection='yes'
    table_SEGMENTS_ZDC = 'SEGMENTS_ZDC_' + str(segment_distance) + 'M'
    table_SEMANTIC_LINKS_SEGMENTS = 'SEMANTIC_LINKS_SEGMENTS_' + str(segment_distance) + 'M'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    sql_query = "select * from " + table_SEGMENTS_ZDC+","+table_SEMANTIC_LINKS_SEGMENTS + " where "+table_SEMANTIC_LINKS_SEGMENTS+".SEGMENT_ID = "+table_SEGMENTS_ZDC+".SEGMENT_ID and SEMANTIC_LINK_ID = "+str(semantic_link_id )
    cursor.execute(sql_query) 
    rows = cursor.fetchall()
    
    cursor.close()
    connect.close()
    return rows

def get_Car_model_datas(car_id):
    driver='{SQL Server}'
    server = '*******************'
    database = '*******************'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    sql_query = "select * from CARS where CAR_ID = "+str(car_id)
    cursor.execute(sql_query) 
    rows = cursor.fetchall()
    while len(rows) < 1 :
        print('CAR_id入力間違いです。確認の上、改めて入力してください:')
        car_id = input()
        sql_query = "select * from CARS where CAR_ID = "+ car_id
        cursor.execute(sql_query)
        rows = cursor.fetchall()
    cursor.close()
    connect.close()
    return rows

def display_folium_map_from_csv(data,path):
    output_folium_file_path = path
    df_output_points = data.iloc[:,1:3]
    folium_map = folium.Map(
        location=[
            df_output_points.at[len(df_output_points) // 2, "LATITUDE"],
            df_output_points.at[len(df_output_points) // 2, "LONGITUDE"],
        ],
        zoom_start=13,
    )
    for index, point in df_output_points.iterrows():
        folium.Marker(location=[point["LATITUDE"], point["LONGITUDE"]]).add_to(
            folium_map
        )
    folium_map.save(output_folium_file_path)
    #webbrowser.open(output_folium_file_path, new=2)

In [ ]:
# KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返すメソッド
def readLinksKML(filename):
    lineSirungTab1 = r"<LineString>"
    lineSirungTab2 = r"</LineString>"
    isLineString = 0
    f = open(filename, "r", encoding="UTF-8")
    datalist = f.readlines()
    GPSlist = [["Latitude", "Longitude"]]
    for data in datalist:
        if isLineString == 1:
            if lineSirungTab2 in data:
                # print('ここまでが対象データ')
                isLineString = 0
            else:
                target_data = data.replace(" ", "").replace(r",0", "")
                if re.match(r"[0-9]+.?[0-9]*,[0-9]+.?[0-9]*", target_data):
                    target_data2 = re.split(",", target_data)
                    appendList = [float(target_data2[1]), float(target_data2[0])]
                    GPSlist.append(appendList)
                    # print(appendList)
        elif isLineString == 0:
            if lineSirungTab1 in data:
                # print('ここからが対象データ')
                isLineString = 1
        else:
            print("よきせぬれーがい")
    GPSlist.remove(["Latitude", "Longitude"])
    return GPSlist


def calcDistance(list_GPS):
    list_lat_long_dist = [
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    ]
    before_lat = 0
    before_long = 0
    dist_sum = 0
    for row in list_GPS:
        if before_lat > 0:
            dist = distance(before_lat, before_long, row[0], row[1])
            appendList = [before_lat, before_long, row[0], row[1], dist]
            list_lat_long_dist.append(appendList)
            dist_sum = dist_sum + dist
            # print(appendList)
        before_lat = row[0]
        before_long = row[1]
    list_lat_long_dist.remove(
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    )
    return list_lat_long_dist, dist_sum

def normalizedCoordinatesGenerator(list_lat_long_dist, normalized_dist):
    normalizedList = [["Latitude", "Longitude"]]
    rest = normalized_dist
    _lat = list_lat_long_dist[0][0]
    _long = list_lat_long_dist[0][1]
    appendList = [_lat, _long]
    normalizedList.append(appendList)
    coordinatesNumber = 1

    for row in list_lat_long_dist:
        row_loop = 0
        if rest > row[4]:
            rest = rest - row[4]
            # print('rest:', str(rest))
        else:
            while rest + normalized_dist * row_loop < row[4]:
                _lat = (row[2] - row[0]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[0]
                _long = (row[3] - row[1]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[1]
                appendList = [_lat, _long]
                normalizedList.append(appendList)
                coordinatesNumber = coordinatesNumber + 1
                row_loop = row_loop + 1
                # print('rest:', str(rest))
            rest = rest - row[4] + normalized_dist * row_loop
    
    return normalizedList, coordinatesNumber

def GPSListGenerator_segment(filename,speed):
    list_GPS = readLinksKML(filename)  # KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返す
    # print(list_GPS)    #(lat, long)
    list_lat_long_dist, dist_sum = calcDistance(list_GPS)
    NORMALIZED_DISTANCE = speed

    normalized_dist = NORMALIZED_DISTANCE

    list_normalized_lat_long, coordinatesNumber = normalizedCoordinatesGenerator(
        list_lat_long_dist, normalized_dist
    )
    
    return list_normalized_lat_long

    

<font size = 6 color =red> ここからは実行↓ </font>

In [ ]:
print('--------------------------------')

print('Semantic_link_idを指定してください(整数)')
semantic_link_id = input()
semantic_link_id = int(semantic_link_id)


kml_filename =  Path(path.join(path.dirname(os.getcwd()), "." +"\Energy_map\INPUT\kml_Semantics\{}.kml".format(str(semantic_link_id))))
json_*******************_filepath =  Path(path.join(path.dirname(os.getcwd()), "." +"\Energy_map\INPUT\*******************\{}.json".format(str(semantic_link_id))))
csv_*******************_filepath =  Path(path.join(path.dirname(os.getcwd()), "." +"\Energy_map\INPUT\*******************\{}.csv".format(str(semantic_link_id))))


print('ACC setting Speedを指定してください(km/h)（整数）：')
Speed = input()
Speed_for_output = int(Speed)
Speed = int(Speed)*0.92 /3.6
print('--------------------------------')
print('Virtual constant velocity runing LOG生成中です,場合によって1min以上かかることがある')
distance_between_segments = Speed
segment_distance = Speed
#read kml and convert it to a DataFrame
#仮想走行ログの座標群を生成する
df_Coordinate = pd.DataFrame(GPSListGenerator_segment(kml_filename,Speed)[1:],columns =GPSListGenerator_segment(kml_filename,Speed)[0])
Coordinate_id_series = list(range(len(df_Coordinate)))

df_Coordinate['Coordinate_id'] = Coordinate_id_series

#To match link_id of roads, a Knn classifier is trained with datas extracted from the df_segment
train_data = df_Coordinate.iloc[:,0:2]
train_label = df_Coordinate.iloc[:,-1]
Classifier = KNeighborsClassifier(n_neighbors=1)
Classifier.fit(train_data,train_label)
#Use trained classifier to match roads
#you can trust this classifier cause of I set n_neighbors to 1,and it must overfitting
#so its accuracy will be 100%

if semantic_link_id in [****************************************************]:
    df_******************* = pd.read_csv(csv_*******************_filepath)
    df_******************* = df_*******************.iloc[:,1:]
else:
    df_******************* = ADASListGenerator_to_df(semantic_link_id ,json_*******************_filepath)
Classifier_result_for_******************* = Classifier.predict(df_*******************.iloc[:,1:3])
#The following DataFrame was created to aggregate the training results and to organize the data
Data_virtual_run = pd.DataFrame({'SEGMENT_ID':[],
                                                'SEGMENT_LENGTH':[],
                                                'START_LATITUDE':[],
                                                'START_LONGITUDE':[],
                                                'END_LATITUDE':[],
                                                'END_LONGITUDE':[],
                                                'START_ADAS_ELEVATION_MILLI_METER':[],
                                                'END_ADAS_ELEVATION_MILLI_METER':[],
                                                'SLOPE_ANGLE_THETA':[],
                                                'COS_THETA':[],
                                                'SIN_THETA':[],
                                                'IS_ADAS_NULL':[],
                                                'IS_GET_ZDC_DATA':[]
                                                })
Data_virtual_run.SEGMENT_ID = df_Coordinate.Coordinate_id.iloc[0:-1]
Data_virtual_run.START_LATITUDE = df_Coordinate.Latitude.iloc[0:-1]
Data_virtual_run.START_LONGITUDE = df_Coordinate.Longitude.iloc[0:-1]
Data_virtual_run.END_LATITUDE = df_Coordinate.Latitude.iloc[1:].reset_index(drop = True)
Data_virtual_run.END_LONGITUDE = df_Coordinate.Longitude.iloc[1:].reset_index(drop = True)


#insert Classified df_******************* to Data_virtual_run
for i in range(len(Classifier_result_for_*******************)):
    index = Data_virtual_run.SEGMENT_ID.isin([Classifier_result_for_*******************[i]])
    Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER.iloc[index] = df_*******************.*******************_Elevation.loc[i]
    Data_virtual_run.IS_ADAS_NULL.iloc[index] = df_*******************.adas_null.loc[i]

#ここからはClassifierできない部分に対して、加重平均などの手法で欠損を埋める
index_after = 0
index_before = 0
index_NOT_nan_list = list(Data_virtual_run[~Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER.isna()].index)
index_nan_list = list(Data_virtual_run[Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER.isna()].index)
IS_GET_ZDC_DATA=[]
for i in range(len(Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER.isna())):
    IS_GET_ZDC_DATA.append(int(~Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER.isna()[i]))
Data_virtual_run['IS_GET_ZDC_DATA'] = IS_GET_ZDC_DATA

before = 0
after = 0
if str(Data_virtual_run.iloc[0]['START_ADAS_ELEVATION_MILLI_METER']) == 'nan':
    Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[0] = Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[index_NOT_nan_list[0]]
    
if str(Data_virtual_run.iloc[-1]['START_ADAS_ELEVATION_MILLI_METER']) == 'nan':
    Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[-1] = Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[index_NOT_nan_list[-1]]
#ここからは加重平均
while Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER.isnull().sum() !=0:
    for i in range(len(Data_virtual_run)-1):
        if str(Data_virtual_run.iloc[i]['START_ADAS_ELEVATION_MILLI_METER'])== 'nan':
            before = Data_virtual_run.iloc[i-1]['START_ADAS_ELEVATION_MILLI_METER']
            index_before = i-1
            #print('index_before:',index_before,' ','before:',before)
            for j in range(i,len(Data_virtual_run)):
                if str(Data_virtual_run.iloc[j]['START_ADAS_ELEVATION_MILLI_METER'])!= 'nan':
                    after = Data_virtual_run.iloc[j]['START_ADAS_ELEVATION_MILLI_METER']
                    index_after = j
                    #print('index_after:',index_after,' ','after:',after)
                    break
            distance_before_to_after = distance(Data_virtual_run.iloc[index_before]['START_LATITUDE'], 
                                                Data_virtual_run.iloc[index_before]['START_LONGITUDE'],
                                                Data_virtual_run.iloc[index_after]['START_LATITUDE'],
                                                Data_virtual_run.iloc[index_after]['START_LONGITUDE'])

            rate_before = distance(Data_virtual_run.iloc[index_before]['START_LATITUDE'], 
                                   Data_virtual_run.iloc[index_before]['START_LONGITUDE'],
                                   Data_virtual_run.iloc[i]['START_LATITUDE'],
                                   Data_virtual_run.iloc[i]['START_LONGITUDE'])/distance_before_to_after
            rate_after = distance(Data_virtual_run.iloc[i]['START_LATITUDE'], 
                                  Data_virtual_run.iloc[i]['START_LONGITUDE'],
                                  Data_virtual_run.iloc[index_after]['START_LATITUDE'],
                                  Data_virtual_run.iloc[index_after]['START_LONGITUDE'])/distance_before_to_after
            #print(before * (1-rate_before) + after*(1-rate_after))
            Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[i] = before * (1-rate_before) + after*(1-rate_after)
            #print('for this loop:',Data_for_inserting.iloc[i]['START_ADAS_ELEVATION_MILLI_METER'])
            break

#次のstartは前のend
Data_virtual_run['END_ADAS_ELEVATION_MILLI_METER'].iloc[1:] = Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[0:-1]
Data_virtual_run['END_ADAS_ELEVATION_MILLI_METER'].iloc[0] = Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[0]
#segment間隔を挿入
Data_virtual_run['SEGMENT_LENGTH'] = Speed

#startとendの標高差を利用して勾配を算出
for i in range(len(Data_virtual_run)):
    delta_ELEVATION = Data_virtual_run['END_ADAS_ELEVATION_MILLI_METER'].iloc[i] - Data_virtual_run['START_ADAS_ELEVATION_MILLI_METER'].iloc[i]
    Data_virtual_run['SLOPE_ANGLE_THETA'].iloc[i] = np.arctan(delta_ELEVATION/(distance_between_segments*1000))
    Data_virtual_run['COS_THETA'].iloc[i] = math.cos(Data_virtual_run['SLOPE_ANGLE_THETA'].iloc[i])
    Data_virtual_run['SIN_THETA'].iloc[i] = math.sin(Data_virtual_run['SLOPE_ANGLE_THETA'].iloc[i])
    if str(Data_virtual_run.iloc[i]['IS_ADAS_NULL'])== 'nan':
        Data_virtual_run['IS_ADAS_NULL'].iloc[i] = 0
Data_virtual_run['IS_ADAS_NULL'] = Data_virtual_run['IS_ADAS_NULL'].astype('int')

print('Virtual constant velocity runing LOG生成完了')
print('--------------------------------')


    
    
########################
#get datas of Car on input car_id from SQL server    
print('--------------------------------')
print('car_idを指定してください(virtual_LEAF2020 is 27,nothing on the car)')
car_id = input()
car_id = int(car_id)
print('車に乗る人や道具の重さの合計を入力してください（kg）')
other_wight = input()
other_wight = int(other_wight)
print('--------------------------------')
datas_from_SQL_server_CARS = get_Car_model_datas(car_id)

df_Car = pd.DataFrame({},columns = ['CAR_ID',
                                    'MODEL',
                                    'BATTERY',
                                    'WEIGHT',
                                    'TIRE_RADIUS',
                                    'REDUCTION_RATIO',
                                    'CD_VAULE',
                                    'FRONTAL_PROJECTED_AREA'])

df_Car.loc[0] = list(datas_from_SQL_server_CARS[0])
df_Car.WEIGHT.loc[0] = df_Car.WEIGHT.loc[0] + other_wight

########################
#calculate Ecolog datas base on segment datas and car data 
print('--------------------------------')
print('VCVR LOGによりECOLOG出力を計算しているฅ^•ω•^ฅ,it will take about 30s~60s,maybe more')
df_ecolog_calculate = pd.DataFrame({'SEGMENT_ID':[],
                                    'LATITUDE':[],
                                    'LONGITUDE':[],
                                    'ELEVATION_MILLI_METER':[],
                                    'SLOPE_ANGLE_THETA':[],
                                    'COS_THETA':[],
                                    'SIN_THETA':[],
                                    'Is_Adas':[]})
df_ecolog_calculate.SEGMENT_ID = Data_virtual_run.SEGMENT_ID
df_ecolog_calculate.LATITUDE = Data_virtual_run.START_LATITUDE
df_ecolog_calculate.LONGITUDE = Data_virtual_run.START_LONGITUDE
df_ecolog_calculate.ELEVATION_MILLI_METER = Data_virtual_run.START_ADAS_ELEVATION_MILLI_METER

# df_ecolog_calculate.SLOPE_ANGLE_THETA = df_segment_in_semantic.SLOPE_ANGLE_THETA
# df_ecolog_calculate.COS_THETA = df_segment_in_semantic.COS_THETA
# df_ecolog_calculate.SIN_THETA = df_segment_in_semantic.SIN_THETA
df_ecolog_calculate.Is_Adas = 1   # 1 means True
df_ecolog_calculate['Speed'] = Speed * 3.6
df_ecolog_calculate['Torque'] = 0.0
df_ecolog_calculate['EnergyByAirResistance'] = 0.0
df_ecolog_calculate['EnergyByRollingResistance'] = 0.0
df_ecolog_calculate['EnergyByClimbingResistance'] = 0.0
df_ecolog_calculate['EnergyByAccResistance'] = 0.0
df_ecolog_calculate['DrivingResistancePower'] = 0.0
df_ecolog_calculate['ConvertLoss'] = 0.0
df_ecolog_calculate['RegeneLoss'] = 0.0
df_ecolog_calculate['RegeneEnergy'] = 0.0
df_ecolog_calculate['LostEnergy'] = 0.0
df_ecolog_calculate['Efficiency'] = 79
df_ecolog_calculate['ConsumedEnergy'] = 0.0

for i in range(len(df_ecolog_calculate)):
    if i > 0:
        delta_ELEVATION = df_ecolog_calculate.ELEVATION_MILLI_METER.iloc[i] - df_ecolog_calculate.ELEVATION_MILLI_METER.iloc[i-1]
        df_ecolog_calculate.SLOPE_ANGLE_THETA.iloc[i] = np.arctan(delta_ELEVATION/(segment_distance*1000))
        df_ecolog_calculate.COS_THETA.iloc[i] =  math.cos(df_ecolog_calculate.SLOPE_ANGLE_THETA.iloc[i])
        df_ecolog_calculate.SIN_THETA.iloc[i] =  math.sin(df_ecolog_calculate.SLOPE_ANGLE_THETA.iloc[i])
        
        airResistancePower = rho * df_Car['CD_VAULE'].iloc[0] * df_Car['FRONTAL_PROJECTED_AREA'].iloc[0] * Speed * Speed / 2 * Speed / 3600 /1000
        rollingResistancePower = myu * df_Car['WEIGHT'].iloc[0] * GravityResistanceCoefficient * df_ecolog_calculate.COS_THETA.iloc[i] * Speed / 3600 /1000
        climbingResistancePower = df_Car['WEIGHT'].iloc[0] * GravityResistanceCoefficient * df_ecolog_calculate.SIN_THETA.iloc[i] * Speed / 3600 /1000
        drivingResistancePower = airResistancePower+rollingResistancePower+climbingResistancePower+0
        df_ecolog_calculate.EnergyByAirResistance.iloc[i] = airResistancePower
        df_ecolog_calculate.EnergyByRollingResistance.iloc[i] = rollingResistancePower
        df_ecolog_calculate.EnergyByClimbingResistance.iloc[i] = climbingResistancePower
        df_ecolog_calculate.EnergyByAccResistance.iloc[i] = 0 #仮想走行では定速なので、加速抵抗がない
        df_ecolog_calculate.DrivingResistancePower.iloc[i] = drivingResistancePower
        torque = 0 
        if Speed > 0:
            torque = int(drivingResistancePower * 1000 * 3600 / df_ecolog_calculate.Speed.iloc[i] * df_Car['TIRE_RADIUS'].iloc[0] / df_Car['REDUCTION_RATIO'].iloc[0])
            df_ecolog_calculate.Torque.iloc[i] = torque
        efficiency = get_efficiency(Speed, torque, df_Car)
        df_ecolog_calculate.Efficiency.iloc[i] = efficiency
        if drivingResistancePower >= 0:
            regeneEnergy = 0
            consumedEnergy = drivingResistancePower / efficiency * 100 / InverterEfficiency
        else:
            regeneEnergy = Calc_RegeneEnergy(drivingResistancePower, Speed, efficiency, df_Car['WEIGHT'].iloc[0])
            consumedEnergy = regeneEnergy
        df_ecolog_calculate.ConsumedEnergy.iloc[i] = consumedEnergy
        df_ecolog_calculate.RegeneEnergy.iloc[i] = regeneEnergy
        if drivingResistancePower >= 0:
            convertLoss = consumedEnergy * (1 - (1 * efficiency / 100 * InverterEfficiency))
            regeneLoss = 0
        else:
            convertLoss = consumedEnergy * ((1 / efficiency * 100 / InverterEfficiency) - 1)
            regeneLoss = drivingResistancePower - regeneEnergy / efficiency * 100 / InverterEfficiency  # こここれで合ってるの？
            #regeneLoss = (drivingResistancePower - regeneEnergy) / efficiency * 100 / InverterEfficiency
            #if regeneLoss > 0:
            #    print(drivingResistancePower, '  ', regeneEnergy, '  ', abs(convertLoss) - regeneLoss)
        lostEnergy = abs(convertLoss) - regeneLoss + airResistancePower + rollingResistancePower
        df_ecolog_calculate.LostEnergy.iloc[i] = lostEnergy
        df_ecolog_calculate.ConvertLoss.iloc[i] = convertLoss
        df_ecolog_calculate.RegeneLoss.iloc[i] = regeneLoss
print('calculation is done,now outputing the mapping image')
print('--------------------------------')


print('now outputing the pure version of ecolog-calculation-result to a csv file in \CSV_ECOLOGtable\pure_version')
output_path_ecolog = Path(rf"{os.getcwd()}\CSV_ECOLOGtable\virtual_run_{semantic_link_id}_acc_setting_speed{Speed_for_output}.csv")
df_ecolog_calculate.to_csv(output_path_ecolog)
print('--------------------------------')

review_path = Path(rf"{os.getcwd()}\FoliumMap\Only_semantics\{semantic_link_id}_acc_setting_speed{Speed_for_output}_{segment_distance}m.html")
display_folium_map_from_csv(df_ecolog_calculate,review_path)

output_path_Energy_map = Path(rf"{os.getcwd()}\xlsx_ECOLOG_EnergyMap\virtual_run_{semantic_link_id}_acc_setting_speed{Speed_for_output}.xlsx")
df_ecolog_Energy_map = df_ecolog_calculate[['LATITUDE','LONGITUDE','ConsumedEnergy']]
df_ecolog_Energy_map.to_excel(output_path_Energy_map)


print('all process are done')

# Credit
- ShichiryにkmlとJsonを処理する関数の部分のcodeと、ECOLOG計算式について、教えていただきました
- 何新に欠損を埋める部分に協力していただきました
- Sogaにエネルギーマップについて親切かつ詳しく解説していただきました、ECOLOG計算式について一緒に議論して再作成しました
- prof.Tommyにsegmentと欠損について大変有用な助言をいただきました